# Requirement:

In [ ]:
geojson == 3.1.0
pymongo == 4.7.2
pandas == 2.1.4
folium == 0.16.0
numpy == 1.26.4
numba == 0.59.0
geopy == 2.4.1
shapely == 2.0.4
geopandas == 0.14.4

In [1]:
import os
import sys

# Add library path to Python path
sys.path.append(os.path.abspath('..'))

# Now import your modules
from library import libStopsPoints, libConnections, libHex
import zipfile
import os
import time
import pymongo as pym
import pandas as pd
import folium
import numpy as np
import requests
import numba

import math
import geopy
from shapely.geometry import Polygon, MultiPolygon, Point, mapping
from geopy.distance import geodesic,great_circle
from folium.plugins import FastMarkerCluster
from datetime import datetime
from geopy.distance import geodesic,great_circle
from pymongo import MongoClient

from pathlib import Path
import geopandas as gpd
import geojson
from IPython.core.display import display, HTML

from library import libStopsPoints, libConnections, libHex
 

# Data:

1. gtfs file of the city.
 ->[repository of gtfs file https://transitfeeds.com/]
2. pbf file of [openstreetmap](openstreetmap.org) extract from of the city/region of interest. ->[repository of osm extract: http://download.geofabrik.de/]

# Url and paths  [**set it!**]

In [3]:
city = 'padam_territory_01'
scenario_name = 'DRT_only_scenario_25' # basecase without drt

# file paths and addresses to be provided:
# population data
shpPath = '../Cities/Padam_terretory_01/shp files/opps.shp'
popCollectionName = "POP"
popField = "pop"

# Study area
# Study area defines the area in which the accessibility calaculation is executes
# study_area_shp_path = 'Paris/shp/studyarea.shp'


directoryGTFS = "../Cities/Padam_terretory_01/gtfs/DRT_only/gtfs_padam_territory_01_DRT_only_scenario_25/"

gtfs_prep_toggle = True
urlMongoDb = "mongodb://localhost:27017/"  # url of the mongodb database
urlMongoDbPop = "mongodb://localhost:27017/" # url of the mongodb database for population
urlServerOsrm = 'http://localhost:5000/' # url of the osrm server of the city

client = MongoClient(urlMongoDb)

gtfsDB = client[str('pta-' + city + '-' +scenario_name)]
popDbName = str('pta-' + city + '-' +scenario_name)

# required parameters
# the date must be in the interval of validity of the gtfs files, check it in the "calendar.txt" and "calendar_dates.txt" files inside the gtfs zip files.
# the day for the computation of accessibility quantitites
days = ['20241201', '20241202']
day_names = ["sunday", "monday"] 




# parameters of walking distance
timeWalk = 15 * 60  # seconds
velocityWalk = 1.31  # m/s ***4.71km/h***
distanceS = timeWalk * velocityWalk

# Parameters that define the resolution and extention of tesselletion and the maximum of the walking time
# grid step of the hexagonal tesselletion in kilometers
gridEdge = 1

# Set check4stops = False if cells / hexagones should be included that do not have stops within.
# Set check4stops = False for preprocessing prior to dynamic mode to gtfs convertion
# Set check4stops = True for citychrone accessibility analysis
check4stops = False

# Start of the computation

### add population data

In [4]:

# shapefile = gpd.read_file(shpPath)
# shapefile.to_file("../Cities/Padam_terretory_01/shp files/opps_for_padam_terretory_01.geojson", driver='GeoJSON')
with open("../Cities/Padam_terretory_01/shp files/opps_for_padam_terretory_01.geojson",encoding="utf8") as f:
    gj = geojson.load(f)
features = gj['features']
gtfsDB["POP"].drop()
gtfsDB["POP"].insert_many(features)


InsertManyResult([ObjectId('6776525082db834e293da0ae'), ObjectId('6776525082db834e293da0af'), ObjectId('6776525082db834e293da0b0'), ObjectId('6776525082db834e293da0b1'), ObjectId('6776525082db834e293da0b2'), ObjectId('6776525082db834e293da0b3'), ObjectId('6776525082db834e293da0b4'), ObjectId('6776525082db834e293da0b5'), ObjectId('6776525082db834e293da0b6'), ObjectId('6776525082db834e293da0b7'), ObjectId('6776525082db834e293da0b8'), ObjectId('6776525082db834e293da0b9'), ObjectId('6776525082db834e293da0ba'), ObjectId('6776525082db834e293da0bb'), ObjectId('6776525082db834e293da0bc'), ObjectId('6776525082db834e293da0bd'), ObjectId('6776525082db834e293da0be'), ObjectId('6776525082db834e293da0bf'), ObjectId('6776525082db834e293da0c0'), ObjectId('6776525082db834e293da0c1'), ObjectId('6776525082db834e293da0c2'), ObjectId('6776525082db834e293da0c3'), ObjectId('6776525082db834e293da0c4'), ObjectId('6776525082db834e293da0c5'), ObjectId('6776525082db834e293da0c6'), ObjectId('6776525082db834e293da0

In [5]:
# load Gtfs data to mongodb database
listOfFile = ['stops.txt', 'routes.txt', 'trips.txt', 'calendar.txt', 'calendar_dates.txt',
              'stop_times.txt']  
libStopsPoints.loadGtfsFile(gtfsDB, directoryGTFS, city, listOfFile)

removing stops  of  padam_territory_01
removing routes  of  padam_territory_01
removing trips  of  padam_territory_01
removing calendar  of  padam_territory_01
removing calendar_dates  of  padam_territory_01
removing stop_times  of  padam_territory_01
gtfs_padam_territory_01_DRT_only_scenario_25.zip
stops.txt -> (708,708)
routes.txt -> (588060,588060)
trips.txt -> (588060,588060)
calendar.txt -> (2,2)
calendar_dates.txt -> (2,2)
stop_times.txt -> (1176120,1176120)


## Fill the database with the connections

In [6]:
#add connections for two different days
libConnections.readConnections(gtfsDB, city, directoryGTFS, days[0], day_names[0], overwrite=True)
libConnections.readConnections(gtfsDB, city, directoryGTFS, days[1], day_names[1], overwrite=False)


number of file in calendar+calendar_dates: 1
in stops: 1

Checking the number of services active in the date selected:
file: gtfs_padam_territory_01_DRT_only_scenario_25.zip 	 total number of active service (in calendar.txt): 1
number of different service_id: 1


file: gtfs_padam_territory_01_DRT_only_scenario_25.zip 	 total number of active service (in calendar_dates.txt): 2
number of different service_id: 1 total number of active services found: 2
number of trips 588000
 gtfs_padam_territory_01_DRT_only_scenario_25.zip
inserting to DB....94030, err 0, err_start 0, err_start_after 0
tot connections 294030
number of file in calendar+calendar_dates: 1
in stops: 1

Checking the number of services active in the date selected:
file: gtfs_padam_territory_01_DRT_only_scenario_25.zip 	 total number of active service (in calendar.txt): 1
number of different service_id: 1


file: gtfs_padam_territory_01_DRT_only_scenario_25.zip 	 total number of active service (in calendar_dates.txt): 2
number 

## remove stops with no connections and add to each stop the pos field

In [7]:
libStopsPoints.removingStopsNoConnections(gtfsDB, city)
libStopsPoints.setPosField(gtfsDB, city)

In [8]:
libConnections.updateConnectionsStopName(gtfsDB, city)

connections deleted 060 pEnd 588060, totC 588060


# Tassel with exagons

### List of all stops

In [9]:
stopsList = libStopsPoints.returnStopsList(gtfsDB, city)

tot stop 708  stop error : 0


## Compute the box that include all stops
The edge of such box are enlarged by distanceS.

In [10]:

display(HTML('<h1>All stops of the public transport present in the gtfs files</h1>'))
bbox = libStopsPoints.boundingBoxStops(stopsList)
libStopsPoints.mapStops(bbox, stopsList)

## Tassel the box with exagons.

In [11]:
hexBin, pointBin = libHex.hexagonalGrid(bbox, gridEdge, gtfsDB['stops'], distanceS, city,check4stops)

In [12]:

libHex.insertPoints(pointBin, city, gtfsDB)


## Find the hex with walkingTime less than timeWalk from a stops

In [13]:

libHex.pointsServed(gtfsDB, stopsList, urlServerOsrm, distanceS, timeWalk, city)

In [14]:
print("Number of hexagons: {0}".format(gtfsDB['points'].count_documents({'served':True, 'city':city})))

Number of hexagons: 249


## Setting field "pos" for points for performance

In [15]:
from library import libHex 
libHex.settingHexsPos(gtfsDB, city)

## Setting Population of Hexagons

In [16]:
# from libHex import setHexsPop
import library
from library import libHex
if urlMongoDbPop != "" and popCollectionName != "":
    clientPop = pym.MongoClient(urlMongoDbPop)
    popDb = clientPop[popDbName]
    popCollection = popDb[popCollectionName]
    libHex.setHexsPop(gtfsDB, popCollection, popField, city)
else:
    print("Population NOT INSERTED!")



# Adding the walking time between stops and points

In [17]:
from library import libStopsPoints 
libStopsPoints.computeNeigh(gtfsDB, urlServerOsrm, distanceS, timeWalk,  city)

/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


# Compute quantities and observable

TimeList is the list of starting time for computing the isochrones

In [18]:
timeList = [7,10,12,17,20]

hStart = timeList[0]*3600

### List of connections

In [19]:
from library import libConnections 
# arrayCC = libConnections.makeArrayConnections(gtfsDB, hStart, city)
arrayCC_day1 = libConnections.makeArrayConnections(gtfsDB, hStart, city, day='20241201')
arrayCC_day2 = libConnections.makeArrayConnections(gtfsDB, hStart, city, day='20241202')

start making connections array
done recover all cc 294030
converted
Num of connection 294030
start making connections array
done recover all cc 294030
converted
Num of connection 294030


### List of list of the points and stops neighbors

In [20]:
from library import libStopsPoints 
arraySP = libStopsPoints.listPointsStopsN(gtfsDB, city)

## Compute accessibility quantities

In [ ]:
import imp
from library import libAccessibility_version2, shortest_path_algo
imp.reload(libAccessibility_version2)
from library.shortest_path_algo import computeAccessibilities
imp.reload(shortest_path_algo)

listAccessibility = ['socialityScore30min']

computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    #gtfsDB['isochrones'].delete_many({'city':city})
    pass
for timeStart in timeList:
    timeStart *= 3600
    hEnd  = timeStart+6600
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
#     computeAccessibilities(city, timeStart, arrayCC, arraySP, gtfsDB, computeIsochrone, timeStart/3600 == timeList[0], listAccessibility=listAccessibility)
    
    computeAccessibilities(
    city, timeStart,hEnd, arrayCC_day1, arraySP, gtfsDB, 
    computeIsochrone, timeStart/3600 == timeList[0], 
    day='20241201', 
    max_travel_time=1800, #30min 
    listAccessibility=listAccessibility
)

    # For second day
    computeAccessibilities(
        city, timeStart,hEnd, arrayCC_day2, arraySP, gtfsDB,
        computeIsochrone, timeStart/3600 == timeList[0],  # Use first=True for each day
        day='20241202',
        max_travel_time=1800, #30min 
        listAccessibility=listAccessibility
    )

Time Isochrone Start: 7.0


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

In [22]:
import imp
from library import libAccessibility, icsa
imp.reload(libAccessibility)
from library.icsa import computeAccessibilities
imp.reload(icsa)

listAccessibility = ['socialityScore1h']

computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    #gtfsDB['isochrones'].delete_many({'city':city})
    pass
for timeStart in timeList:
    timeStart *= 3600
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
#     computeAccessibilities(city, timeStart, arrayCC, arraySP, gtfsDB, computeIsochrone, timeStart/3600 == timeList[0], listAccessibility=listAccessibility)
    
    computeAccessibilities(
    city, timeStart, arrayCC_day1, arraySP, gtfsDB, 
    computeIsochrone, timeStart/3600 == timeList[0], 
    day='20241201', 
    max_travel_time=3600, #1H 
    listAccessibility=listAccessibility
)

    # For second day
    computeAccessibilities(
        city, timeStart, arrayCC_day2, arraySP, gtfsDB,
        computeIsochrone, timeStart/3600 == timeList[0],  # Use first=True for each day
        day='20241202',
        max_travel_time=3600, #1H 
        listAccessibility=listAccessibility
    )

Time Isochrone Start: 7.0


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

Time Isochrone Start: 10.0h_20241202: 4.4, time to finish: 0.0h, 0.0mmm


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

...

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

Time Isochrone Start: 20.0h_20241202: 4.4, time to finish: 0.0h, 0.0mmm


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

In [29]:
import imp
from library import libAccessibility_version2, shortest_path_algo
imp.reload(libAccessibility_version2)
from library.shortest_path_algo import computeAccessibilities
imp.reload(shortest_path_algo)

listAccessibility = ['socialityScore20min']

computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    #gtfsDB['isochrones'].delete_many({'city':city})
    pass
for timeStart in timeList:
    timeStart *= 3600
    hEnd  = timeStart+6600
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
#     computeAccessibilities(city, timeStart, arrayCC, arraySP, gtfsDB, computeIsochrone, timeStart/3600 == timeList[0], listAccessibility=listAccessibility)
    
    computeAccessibilities(
    city, timeStart,hEnd, arrayCC_day1, arraySP, gtfsDB, 
    computeIsochrone, timeStart/3600 == timeList[0], 
    day='20241201', 
    max_travel_time=1200, #20min 
    listAccessibility=listAccessibility
)

    # For second day
    computeAccessibilities(
        city, timeStart,hEnd, arrayCC_day2, arraySP, gtfsDB,
        computeIsochrone, timeStart/3600 == timeList[0],  # Use first=True for each day
        day='20241202',
        max_travel_time=1200, #20min 
        listAccessibility=listAccessibility
    )

Time Isochrone Start: 7.0


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

Time Isochrone Start: 10.020min_20241202: 4.5, time to finish: 0.0h, 0.0mmm


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

...

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

Time Isochrone Start: 20.020min_20241202: 4.5, time to finish: 0.0h, 0.0mmm


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

In [28]:
import imp
from library import libAccessibility, icsa
imp.reload(libAccessibility)
from library.icsa import computeAccessibilities
imp.reload(icsa)

listAccessibility = ['socialityScore10min']

computeIsochrone = False
if 'isochrones' in gtfsDB.list_collection_names():
    #gtfsDB['isochrones'].delete_many({'city':city})
    pass
for timeStart in timeList:
    timeStart *= 3600
    print( 'Time Isochrone Start: {0}'.format(timeStart/3600,))
#     computeAccessibilities(city, timeStart, arrayCC, arraySP, gtfsDB, computeIsochrone, timeStart/3600 == timeList[0], listAccessibility=listAccessibility)
    
    computeAccessibilities(
    city, timeStart, arrayCC_day1, arraySP, gtfsDB, 
    computeIsochrone, timeStart/3600 == timeList[0], 
    day='20241201', 
    max_travel_time=600, #10min 
    listAccessibility=listAccessibility
)

    # For second day
    computeAccessibilities(
        city, timeStart, arrayCC_day2, arraySP, gtfsDB,
        computeIsochrone, timeStart/3600 == timeList[0],  # Use first=True for each day
        day='20241202',
        max_travel_time=600, #10min 
        listAccessibility=listAccessibility
    )

Time Isochrone Start: 7.0


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

Time Isochrone Start: 10.010min_20241202: 4.5, time to finish: 0.0h, 0.0mm


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

...

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

Time Isochrone Start: 20.010min_20241202: 4.5, time to finish: 0.0h, 0.0mm


/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov

/home/medourahou1/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/medourahou1/.local/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/medourahou1/.local/lib/python3.10/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-ov